In [1]:
#default_exp data.datasets.on

In [2]:
#export
from dataclasses import dataclass
from datetime import timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
from pandas.tseries.frequencies import to_offset

from nixtla.data.datasets.utils import download_file, Info, TimeSeriesDataclass
from nixtla.data.ontsdataset import TimeSeriesDataset

In [5]:
#export
def load_benchmark_data(root_dir, test_date):
    demanda_final = pd.read_csv(f'{root_dir}/Demanda_Final.csv')
    demanda_final.columns = ['ds','y']
    demanda_final['unique_id'] = 'demanda_final'
    demanda_final['ds'] = pd.to_datetime(demanda_final['ds'])
    demanda_final=demanda_final[demanda_final['ds']<'2020-12-01']

    ejecutado = pd.read_csv(f'{root_dir}/Despacho_Reprograma.csv')
    ejecutado = ejecutado[['DATE_TIME','EJECUTADO']]
    ejecutado.columns = ['ds','ejecutado']
    ejecutado['ds'] = pd.to_datetime(ejecutado['ds'])
    ejecutado=ejecutado[ejecutado['ds']<'2020-12-01']

    data = demanda_final.merge(ejecutado, how='outer', indicator=True)
    data[['ejecutado']] = data[['ejecutado']].fillna(method='ffill') #bfill for first row
    #data[['ejecutado']] = data[['ejecutado']].interpolate().fillna(method='bfill') #bfill for first row
    data = data[['unique_id','ds','y','ejecutado']]

    data['hour'] = data['ds'].dt.hour
    dummies = pd.get_dummies(data['hour'],prefix='hour')
    data = pd.concat((data,dummies), axis=1)

    data['day'] = data['ds'].dt.dayofweek
    dummies = pd.get_dummies(data['day'],prefix='day')
    data = pd.concat((data,dummies), axis=1)

    dummies_cols = [col for col in data if (col.startswith('day') or col.startswith('hour_'))]
    X_t_df = data[['unique_id','ds','ejecutado'] + dummies_cols]
    y_df = data[['unique_id','ds','y']]

    y_df_insample = y_df[y_df['ds'] < test_date].reset_index(drop=True)
    y_df_outsample = y_df[y_df['ds'] >= test_date].reset_index(drop=True)

    X_t_insample_df = X_t_df[X_t_df['ds'] < test_date].reset_index(drop=True)
    X_t_outsample_df = X_t_df[X_t_df['ds'] >= test_date].reset_index(drop=True)

    return y_df_insample, X_t_insample_df, y_df_outsample, X_t_outsample_df


def load_on_data(root_dir, test_date):
    # Declare future variables
    f_cols = ['PD_0','PS_0','day_0','day_1','day_2','day_3','day_4','day_5','day_6',
              'hour_0', 'hour_1','hour_2','hour_3','hour_4','hour_5','hour_6','hour_7','hour_8',
              'hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16',
              'hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',
              'temperature_0', 'temperature_1', 'temperature_2', 'temperature_3', 'temperature_4',
              'pressure_0', 'pressure_1', 'pressure_2', 'pressure_3', 'pressure_4', 'humidity_0',
              'humidity_1', 'humidity_2', 'humidity_3', 'humidity_4', 'wind_speed_0', 'wind_speed_1',
              'wind_speed_2', 'wind_speed_3', 'wind_speed_4', 'rain_0', 'rain_1', 'rain_2', 'rain_3', 'rain_4']

    # Demanda Final
    demanda_final = pd.read_csv(f'{root_dir}/Demanda_Final.csv')
    demanda_final.columns = ['ds','y']
    demanda_final['unique_id'] = 'demanda_final'
    demanda_final['ds'] = pd.to_datetime(demanda_final['ds'])
    demanda_final = demanda_final[demanda_final['ds']<'2020-12-01']
    demanda_final = demanda_final[demanda_final['ds']>='2017-10-07'].reset_index(drop=True)

    # Ejecutado
    ejecutado = pd.read_csv(f'{root_dir}/Despacho_Reprograma.csv')
    ejecutado = ejecutado[['DATE_TIME','EJECUTADO']]
    ejecutado.columns = ['ds','ejecutado']
    ejecutado['ds'] = pd.to_datetime(ejecutado['ds'])
    ejecutado = ejecutado[ejecutado['ds']<'2020-12-01']
    ejecutado = ejecutado[ejecutado['ds']>='2017-10-07'].reset_index(drop=True)

    # Train data (ON exogenous variables)
    train_data = pd.read_csv(f'{root_dir}/Training_Data.csv')
    train_data = train_data.rename(columns={'CURRENT_TIME':'ds'})
    train_data['ds'] = pd.to_datetime(train_data['ds'])
    train_data = train_data[train_data['ds']<'2020-12-01']
    train_data = train_data[train_data['ds']>='2017-10-07'].reset_index(drop=True)

    # Historic Weather
    data_weather = pd.read_csv(f'{root_dir}/HistoricalWeather.csv')
    data_weather['dt_iso'] = data_weather['dt_iso'].str[:-10]
    data_weather['dt_iso'] = pd.to_datetime(data_weather['dt_iso'])
    data_weather = data_weather[data_weather['dt_iso']>='2017-10-07'].reset_index(drop=True)
    data_weather = data_weather[['dt_iso', 'city_name', 'temp', 'feels_like', 'rain_1h', 'pressure', 'humidity', 'wind_speed', 'clouds_all']]
    data_weather = data_weather.drop_duplicates(subset=['dt_iso', 'city_name']).reset_index(drop=True)
    data_weather = data_weather.pivot(index='dt_iso', columns='city_name').reset_index()
    data_weather.columns = ['_'.join(col).strip() for col in data_weather.columns.values]
    data_weather = data_weather.rename(columns = {'dt_iso_':'ds'})
    data_weather = data_weather.fillna(0) # Fill rain nans with 0

    # Forecast Weather
    data_f_weather = pd.read_csv(f'{root_dir}/Historical ForecastWeather.csv')
    # Between 1 and 21,600 takes next 6h of forecasts. Always uses last available forecast for each ds
    data_f_weather = data_f_weather[(data_f_weather['slice dt unixtime']-data_f_weather['forecast dt unixtime']). \
                                    between(1, 21600)].reset_index(drop=True)
    data_f_weather['forecast dt iso'] = data_f_weather['forecast dt iso'].str[:-10]
    data_f_weather['forecast dt iso'] = pd.to_datetime(data_f_weather['forecast dt iso'])
    data_f_weather = data_f_weather[data_f_weather['forecast dt iso']>='2017-10-07'].reset_index(drop=True)
    data_f_weather = data_f_weather.rename(columns = {'forecast dt iso':'ds'})
    data_f_weather['city'] = data_f_weather.groupby(['lat']).ngroup()
    data_f_weather['city'] = data_f_weather['city'].astype(str)
    data_f_weather = data_f_weather[['ds', 'city', 'temperature', 'pressure', 'humidity', 'wind_speed', 'rain']]
    data_f_weather = data_f_weather.drop_duplicates(subset=['ds', 'city']).reset_index(drop=True)
    data_f_weather = data_f_weather.pivot(index='ds', columns='city').reset_index()
    data_f_weather.columns = ['_'.join(col).strip() for col in data_f_weather.columns.values]
    data_f_weather = data_f_weather.rename(columns = {'ds_':'ds'})
    data_f_weather = data_f_weather.fillna(0) # Fill rain nans with 0

    # Merge
    data = demanda_final.merge(ejecutado, how='left', indicator=False) #ejecutado needs to be first exogenous variable
    data = data.merge(train_data, how='left', indicator=False)
    data[['ejecutado']] = data[['ejecutado']].fillna(method='ffill') #ffill to avoid leakage from immediate next ejecutado
    data = data.merge(data_weather, how='left', on='ds')
    data = data.merge(data_f_weather, how='left', on='ds')
    data = data.fillna(method='ffill') # ffill 15-mintues with last available forecast

    data['hour'] = data['ds'].dt.hour
    dummies = pd.get_dummies(data['hour'],prefix='hour')
    data = pd.concat((data,dummies), axis=1)

    data['day'] = data['ds'].dt.dayofweek
    dummies = pd.get_dummies(data['day'],prefix='day')
    data = pd.concat((data,dummies), axis=1)

    dummies_cols = [col for col in data if (col.startswith('day') or col.startswith('hour_'))]

    X_t_df = data.drop(columns=['hour','day','y'])
    y_df = data[['unique_id','ds','y']]

    Y_insample_df = y_df[y_df['ds'] < test_date].reset_index(drop=True)
    Y_outsample_df = y_df[y_df['ds'] >= test_date].reset_index(drop=True)

    X_insample_df = X_t_df[X_t_df['ds'] < test_date].reset_index(drop=True)
    X_outsample_df = X_t_df[X_t_df['ds'] >= test_date].reset_index(drop=True)

    return Y_insample_df, X_insample_df, Y_outsample_df, X_outsample_df, f_cols

In [6]:
Y_insample_df, X_insample_df, Y_outsample_df, X_outsample_df, f_cols = load_on_data('2020-11-1')

In [7]:
Y_insample_df.head()

,unique_id,ds,y
0,demanda_final,2017-10-07 00:00:00,5545.98338
1,demanda_final,2017-10-07 00:15:00,5448.66753
2,demanda_final,2017-10-07 00:30:00,5392.92989
3,demanda_final,2017-10-07 00:45:00,5371.67098
4,demanda_final,2017-10-07 01:00:00,5296.58436


In [8]:
X_insample_df.head()

,ds,unique_id,ejecutado,DECIMAL_HOUR,DAY_OF_YEAR,DAY_OF_WEEK,IS_WEEKDAY,IS_HOLIDAY,L_1,L_2,L_3,L_4,L_5,L_6,L_7,PE_1,PD_1,PS_1,PE_7,PD_7,PS_7,PD_0,PS_0,P_1,P_2,P_3,P_4,P_5,P_6,P_7,AVG_1,AVG_2,AVG_3,AVG_4,AVG_5,AVG_6,AVG_7,MOV_7,MOV_14,DELTA_1,DELTA_7,PEAK_TIME_1,PEAK_TIME_2,PEAK_TIME_3,PEAK_TIME_4,PEAK_TIME_5,PEAK_TIME_6,PEAK_TIME_7,PE_PP1,PE_PP2,PE_PP3,PE_PP4,PE_PP5,PE_PP6,PE_PP7,PE_PP8,PE_PP9,PE_PP10,PE_PP11,PE_PP12,PE_PP13,PE_PP14,PE_PP15,PE_PP16,PE_PP17,PE_PP18,PE_PP19,PE_PP20,PE_PP21,PE_PP22,PE_PP23,PE_PP24,PE_PP25,PE_PP26,PE_PP27,PE_PP28,PE_PP29,PE_PP30,PE_PP31,PE_PP32,PE_PP33,PE_PP34,PE_PP35,PE_PP36,PE_PP37,PE_PP38,PE_PP39,PE_PP40,PE_PP41,PE_PP42,PE_PP43,PE_PP44,PE_PP45,PE_PP46,PE_PP47,PE_PP48,PD_PP1,PD_PP2,PD_PP3,PD_PP4,PD_PP5,PD_PP6,PD_PP7,PD_PP8,PD_PP9,PD_PP10,PD_PP11,PD_PP12,PD_PP13,PD_PP14,PD_PP15,PD_PP16,PD_PP17,PD_PP18,PD_PP19,PD_PP20,PD_PP21,PD_PP22,PD_PP23,PD_PP24,PD_PP25,PD_PP26,PD_PP27,PD_PP28,PD_PP29,PD_PP30,PD_PP31,PD_PP32,PD_PP33,PD_PP34,PD_PP35,PD_PP36,PD_PP37,PD_PP38,PD_PP39,PD_PP40,PD_PP41,PD_PP42,PD_PP43,PD_PP44,PD_PP45,PD_PP46,PD_PP47,PD_PP48,PS_PP1,PS_PP2,PS_PP3,PS_PP4,PS_PP5,PS_PP6,PS_PP7,PS_PP8,PS_PP9,PS_PP10,PS_PP11,PS_PP12,PS_PP13,PS_PP14,PS_PP15,PS_PP16,PS_PP17,PS_PP18,PS_PP19,PS_PP20,PS_PP21,PS_PP22,PS_PP23,PS_PP24,PS_PP25,PS_PP26,PS_PP27,PS_PP28,PS_PP29,PS_PP30,PS_PP31,PS_PP32,PS_PP33,PS_PP34,PS_PP35,PS_PP36,PS_PP37,PS_PP38,PS_PP39,PS_PP40,PS_PP41,PS_PP42,PS_PP43,PS_PP44,PS_PP45,PS_PP46,PS_PP47,PS_PP48,DELTA_PP1,DELTA_PP2,DELTA_PP3,DELTA_PP4,DELTA_PP5,DELTA_PP6,DELTA_PP7,DELTA_PP8,DELTA_PP9,DELTA_PP10,DELTA_PP11,DELTA_PP12,DELTA_PP13,DELTA_PP14,DELTA_PP15,DELTA_PP16,DELTA_PP17,DELTA_PP18,DELTA_PP19,DELTA_PP20,DELTA_PP21,DELTA_PP22,DELTA_PP23,DELTA_PP24,DELTA_PP25,DELTA_PP26,DELTA_PP27,DELTA_PP28,DELTA_PP29,DELTA_PP30,DELTA_PP31,DELTA_PP32,DELTA_PP33,DELTA_PP34,DELTA_PP35,DELTA_PP36,DELTA_PP37,DELTA_PP38,DELTA_PP39,DELTA_PP40,DELTA_PP41,DELTA_PP42,DELTA_PP43,DELTA_PP44,DELTA_PP45,DELTA_PP46,DELTA_PP47,DELTA_PP48,temp_Ancash,temp_Arequipa,temp_Cusco,temp_Ica,temp_Lima,feels_like_Ancash,feels_like_Arequipa,feels_like_Cusco,feels_like_Ica,feels_like_Lima,rain_1h_Ancash,rain_1h_Arequipa,rain_1h_Cusco,rain_1h_Ica,rain_1h_Lima,pressure_Ancash,pressure_Arequipa,pressure_Cusco,pressure_Ica,pressure_Lima,humidity_Ancash,humidity_Arequipa,humidity_Cusco,humidity_Ica,humidity_Lima,wind_speed_Ancash,wind_speed_Arequipa,wind_speed_Cusco,wind_speed_Ica,wind_speed_Lima,clouds_all_Ancash,clouds_all_Arequipa,clouds_all_Cusco,clouds_all_Ica,clouds_all_Lima,temperature_0,temperature_1,temperature_2,temperature_3,temperature_4,pressure_0,pressure_1,pressure_2,pressure_3,pressure_4,humidity_0,humidity_1,humidity_2,humidity_3,humidity_4,wind_speed_0,wind_speed_1,wind_speed_2,wind_speed_3,wind_speed_4,rain_0,rain_1,rain_2,rain_3,rain_4,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,2017-10-07 00:00:00,demanda_final,5522.52359,0.00,280,7,0,0,5427.81282,5444.23863,5382.13134,5297.62816,5003.93277,5469.80380,5543.54757,5401.05189,5567.13782,5416.46841,5533.78500,5495.96699,5435.83580,5559.55969,5447.38701,6285.68004,6063.81402,6274.26737,6186.15479,6153.28317,6017.21126,6227.70716,5812.529571,5683.086984,5659.550477,5552.33364,5455.458901,5137.682206,5641.027461,5563.095606,5595.209252,473.150469,586.679699,19.0,21.75,18.75,19.75,19.0,20.25,20.75,5705.92737,5778.30200,5959.79329,6127.81847,6179.83933,6226.70145,6226.43267,6240.99027,6227.01880,6271.91587,6252.88890,5948.89415,6013.53241,6113.22254,6050.63703,6186.22110,6217.96117,6229.67571,6174.85669,6099.79195,6047.35638,6089.84764,6113.87500,6205.50876,6296.94876,6194.11048,6156.32352,6164.34407,6041.92704,5999.30681,5910.66906,5721.48965,5648.94435,5581.98574,5458.88953,5342.26518,5210.87922,5194.31368,5117.02403,5015.20287,5011.75982,5025.32881,4917.43509,5017.27217,5066.98034,5104.04557,52